# 1. Imports and data retrieval

In [ ]:
import import_ipynb
import LSH

#TensorFlow
import tensorflow as tf
import tensorflow_hub as hubA

#Sklearn
from sklearn.preprocessing import normalize

#Utilities
import os
import math
from os import path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from pathlib import Path
import time
from PIL import Image
import base64


# Loading the datasets
! gdown --id 1PkbSO8Jn5QEV2ufrD5fwL848xirBs0mW
! gdown --id 1FpSx5nDARAUqe8NPgisDXHzE8B50TD0p

train = pd.read_csv('training_set.csv')
distractor = pd.read_csv('distractor.csv')
  
! rm training_set.csv
! rm distractor.csv 

# loading descriptors computed with the finetuned network
! gdown --id 1bHWVBTYMhKyDqDinkv4QORy0eq_ih237
! gdown --id 1YbNMPkxgKVvFEIq0_fTr83X1LGgNOIh_

descriptors_train_ft = np.load("train_descriptors.npy")
descriptors_distractor_ft = np.load("distractor.npy")

#Remove the files from runtime
! rm train_descriptors.npy
! rm test_descriptors.npy
! rm valid_descriptors.npy
! rm distractor.npy

Downloading...
From: https://drive.google.com/uc?id=1SbHWZBmgEj6PIPthx8wU6ZHPl-nWPG4Z
To: /content/100-bird-species.zip
100% 1.25G/1.25G [00:16<00:00, 77.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-CRBzlwToF2YkiZ9C3Vz6nIvR_AmcS8N
To: /content/DistractormY2pWptmp
100% 3.08G/3.08G [01:06<00:00, 46.1MB/s]
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=1a9e54ac6fdb8811086174a3e5e9572058450896ac88d80469bfea421a163cae
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/10pkR_5XDRQmC8fLU1wQPDHdJpKrJd8At/MIRCVProject/Notebooks
importing Jupyter notebook from LSH.ipynb
/content
Downloading...
From: https://drive.google.com/uc?id=1hOwriTnQGvdXD4oEia6LSToQb9PWRKc_
To: /content/training_set.csv
100% 2.24M/2.24M [00:00<00:00, 35.9MB/s]
Downloading...
From: https://drive.google.com/uc?

In [ ]:
# Download the best performing finetuned model
!gdown --id 1_05XncDr_0PgcbxcvKqmvbE6LxhhzCIG
model = tf.keras.models.load_model('inception_classifier_finetune_block9_0_249_299x299_reg.h5')
extractor = tf.keras.models.Model(inputs=model.inputs, outputs=model.get_layer('classifier_hidden2').output)

Downloading...
From: https://drive.google.com/uc?id=1EMdlLZRzYmB0LfK3TKf-PFEJPqjg4gBI
To: /content/inception_classifier_finetune_block9_0_249_299x299_reg.h5
100% 210M/210M [00:02<00:00, 97.2MB/s]
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 299, 299, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 classifier_hidden1 (Dense)  (None, 1024)              2098176   
                                                                 
 classifier_hidden2 (Dense)  (None, 512)               524800    
                                                                 
 dense_8 (Dense)             (None, 325)               166725    
                                                        

# 2. Building the index

In [ ]:
# concatenating the training descriptors and the distractor descriptors, and binarizing the resulting database
database_ft = np.concatenate((descriptors_train_ft, descriptors_distractor_ft))
binarized_ft, mean_vector_ft = LSH.binarize(database_ft)

  0%|          | 0/65465 [00:00<?, ?it/s]

In [ ]:
# Index the binarized database with the
# best parameters resulting from the experiments. 
# (Binary indexing with 7 hash functions and 5 h function for each hash function)
G = LSH.index_database(7, 5, binarized_ft, binary=True)

  0%|          | 0/65465 [00:00<?, ?it/s]

# 3. Utilities

In [ ]:
def compute_knn(G, descriptor, k, dataset, binary=False):
    """
    Given the descriptor of an image, returns the k indexes of dataset that 
    correspond to the closest neighbors.

    Parameters:
    G           - the index (list of hash functions)
    descriptor  - the descriptor of the query image
    k           - the number of neighbors to return
    dataset     - the dataset of images, with their paths and labels
    binary      - whether to binarize the descriptor before searching the index
    """
  return LSH.get_knn(G, descriptor, k, dataset, binary)

In [ ]:
def extract_descriptor(path):
  """
  Given the path of an image, returns its descriptor.

  Parameters:
  path - the path of the image in the runtime.
  """
  image_data = tf.io.read_file(path)   # read image file
  image = tf.image.decode_image(image_data, channels=3, expand_animations=False)  # decode image data as RGB (do not load whole animations, i.e., GIFs)
  image = tf.image.resize(image, (299, 299)) # resize
  
  image = np.array(image)
  image = np.expand_dims(image, axis=0)   # add batch dimension
  image = image.astype(np.float32)

  image = tf.keras.applications.inception_v3.preprocess_input(image) # preprocess the array for inception_v3
  image_descriptor = extractor.predict(image) # extract the descriptor
  image_descriptor = image_descriptor.squeeze()   # tf.Tensor to numpy array
  return image_descriptor

# 4. Exposed API

In [ ]:
def get_knn_images(query):
  """
  Given the base 64 representation of an image, returns an array of [label, base64]
  that contains for each neighbor image its label and its base 64 representation.

  Parameters:
  query - the base 64 representation of the query image.
  """
  results = []
  # decode the image and save it in a temporary file
  image_64_decode = base64.decodestring(query) 
  path = 'tmp_' + str(time.time()) + str(query[:5]).replace("/", "0") + '.png'
  image_result = open(path, 'wb') # create a writable image and write the decoding result
  image_result.write(image_64_decode)

  # extract the descriptor of the image and binarize it
  descriptor = extract_descriptor(path)
  binarized_desc = LSH.binarize(np.array([descriptor]), mean_vector_ft)

  # remove the temporary file
  if os.path.exists(path):
    os.remove(path)

  # compute the 10 nearest neighbors  
  result = compute_knn(G, np.array(binarized_desc[0]), 10, binarized_ft, True)

  # for each neighbor image, retrieve the image path and the label, then compute
  # the base 64 representation and append both the label and the representation
  # to the array of results.
  for i in result:
    image_path = database.iloc[i].filepaths
    image_label =  database.iloc[i].labels
    with open(image_path, "rb") as image_file:
      encoded_string = base64.b64encode(image_file.read())
      results.append([image_label, encoded_string.decode()])
  return results